In [9]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from sklearn.model_selection import train_test_split

import gensim

In [10]:
# tokenized_df = train_df.copy()

# # Iterate over each column
# for column in train_df.columns:
#     # Check if the column is not 'overall'
#     if column != 'overall':
#         # Tokenize values in the column
#         tokenized_values = train_df[column].apply(lambda x: word_tokenize(x))
#         # Add tokenized values to the new DataFrame
#         tokenized_df[column] = tokenized_values

# tokenized_df

# word2vec_model = gensim.models.Word2Vec(subset_df["tokens"].tolist(), min_count=5, window=9, vector_size=100)

In [11]:
train_df_ss = train_df[['overall', 'ssprocessing_text']].copy()
train_df_ss

,overall,ssprocessing_text
0,5,love glitter pen sparkl delight page brilliant...
1,5,work well machin use most cone
2,5,great assort color though lot pink mix still c...
3,5,look
4,5,make 400 bird hospit month
...,...,...
370850,5,love die make great background card
370851,5,love daric emboss folder darci folder reason e...
370852,5,order add earthi marker previous order want br...
370853,4,made perfect white color blend tini amount yel...


In [12]:
train_df_ps = train_df[['overall', 'psprocessing_text']].copy()
train_df_ps

,overall,psprocessing_text
0,5,love glitter pen sparkl delight page brilliant...
1,5,work well machin use mostli cone
2,5,great assort color though lot pink mix still c...
3,5,look
4,5,make 400 bird hospit month
...,...,...
370850,5,love die make great background card
370851,5,love daric emboss folder darci folder reason e...
370852,5,order add earthi marker previous order want br...
370853,4,made perfect white color blend tini amount yel...


In [13]:
train_df_lem = train_df[['overall', 'lemprocessing_text']].copy()
train_df_lem

,overall,lemprocessing_text
0,5,love glitter pen sparkle delightfully page bri...
1,5,work well machine use mostly cone
2,5,great assortment color though lot pink mix sti...
3,5,looking
4,5,make 400 bird hospital month
...,...,...
370850,5,love dy make great background card
370851,5,love darice embossing folder darcie folder rea...
370852,5,ordered add earthy marker previously ordered w...
370853,4,made perfect white color blending tiny amount ...


In [14]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

def early_stopping(monitor='val_loss', min_delta=0, patience=5, mode='auto'):
    return EarlyStopping(monitor=monitor, min_delta=min_delta, patience=patience, mode=mode)

# Function to create step decay learning rate scheduler
def step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5):
    def scheduler(epoch):
        return initial_lr * np.power(drop_factor, np.floor((1 + epoch) / epochs_drop))
    return scheduler

### LSTM Lemmatising

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Split the data into features and target labels
X = train_df['lemprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model
def LSTM_model(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model
lstm_model = LSTM_model(max_len, vocab_size, embedding_size)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
4622/4622 [==============================] - 1119s 241ms/step - loss: 0.6100 - accuracy: 0.7881 - val_loss: 0.5668 - val_accuracy: 0.7990 - lr: 0.0010
Epoch 2/10
4622/4622 [==============================] - 1148s 248ms/step - loss: 0.5404 - accuracy: 0.8072 - val_loss: 0.5633 - val_accuracy: 0.7986 - lr: 0.0010
Epoch 3/10
4622/4622 [==============================] - 1151s 249ms/step - loss: 0.4998 - accuracy: 0.8227 - val_loss: 0.5666 - val_accuracy: 0.8043 - lr: 0.0010
Epoch 4/10
4622/4622 [==============================] - 1153s 249ms/step - loss: 0.4582 - accuracy: 0.8379 - val_loss: 0.5706 - val_accuracy: 0.8033 - lr: 0.0010
Epoch 5/10
4622/4622 [==============================] - 1238s 268ms/step - loss: 0.4003 - accuracy: 0.8597 - val_loss: 0.6219 - val_accuracy: 0.7998 - lr: 5.0000e-04
Epoch 6/10
4622/4622 [==============================] - 1159s 251ms/step - loss: 0.3719 - accuracy: 0.8704 - val_loss: 0.6506 - val_accuracy: 0.7981 - lr: 5.0000e-04
Epoch 7/10
2311/2311

In [25]:
test_df['Review'] = test_df['Review'].fillna('')
X_test = test_df['Review'] 
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Predict ratings
predicted_labels = lstm_model.predict(X_test_padded)
predicted_ratings = predicted_labels.argmax(axis=1)

# Decode numerical labels back to original categories
predicted_sentiments = label_encoder.inverse_transform(predicted_ratings)

# Combine predictions with IDs
predictions_df = pd.DataFrame({'id': test_df['id'], 'overall': predicted_sentiments})

# Save predictions
predictions_df.to_csv('lstm_lem1.csv', index=False)

# Combine predictions with IDs
tm_df = pd.DataFrame({'id': test_df['id'], 'Review': test_df['Review'], 'overall': predicted_sentiments})

# Save predictions
tm_df.to_csv('lstm_tm.csv', index=False)



3864/3864 [==============================] - 154s 40ms/step


### LSTM with Porter Stemmer

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Split the data into features and target labels
X = train_df['psprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model
def LSTM_model_ps(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model
lstm_model_ps = LSTM_model_ps(max_len, vocab_size, embedding_size)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model_ps.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_ps.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
4622/4622 [==============================] - 1156s 249ms/step - loss: 0.6125 - accuracy: 0.7868 - val_loss: 0.5730 - val_accuracy: 0.7950 - lr: 0.0010
Epoch 2/10
4622/4622 [==============================] - 1058s 229ms/step - loss: 0.5505 - accuracy: 0.8039 - val_loss: 0.5668 - val_accuracy: 0.7982 - lr: 0.0010
Epoch 3/10
4622/4622 [==============================] - 1063s 230ms/step - loss: 0.5153 - accuracy: 0.8164 - val_loss: 0.5658 - val_accuracy: 0.8018 - lr: 0.0010
Epoch 4/10
4622/4622 [==============================] - 1065s 230ms/step - loss: 0.4829 - accuracy: 0.8294 - val_loss: 0.5688 - val_accuracy: 0.8021 - lr: 0.0010
Epoch 5/10
4622/4622 [==============================] - 1063s 230ms/step - loss: 0.4342 - accuracy: 0.8469 - val_loss: 0.5957 - val_accuracy: 0.8035 - lr: 5.0000e-04
Epoch 6/10
4622/4622 [==============================] - 1095s 237ms/step - loss: 0.4103 - accuracy: 0.8560 - val_loss: 0.6262 - val_accuracy: 0.7983 - lr: 5.0000e-04
Epoch 7/10
4622/4622

In [24]:
test_df['Review'] = test_df['Review'].fillna('')
X_test = test_df['Review'] 
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Predict ratings
predicted_labels = lstm_model_ps.predict(X_test_padded)
predicted_ratings = predicted_labels.argmax(axis=1)

# Decode numerical labels back to original categories
predicted_sentiments = label_encoder.inverse_transform(predicted_ratings)

# Combine predictions with IDs
predictions_df = pd.DataFrame({'id': test_df['id'], 'overall': predicted_sentiments})

# Save predictions
predictions_df.to_csv('lstm_ps.csv', index=False)

# Combine predictions with IDs
tm_df = pd.DataFrame({'id': test_df['id'], 'Review': test_df['Review'], 'overall': predicted_sentiments})

# Save predictions
tm_df.to_csv('lstm_ps_tm.csv', index=False)



3864/3864 [==============================] - 135s 35ms/step


### LSTM with Snowball Stemmer

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Split the data into features and target labels
X = train_df['ssprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model
def LSTM_model_ss(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model
lstm_model_ss = LSTM_model_ss(max_len, vocab_size, embedding_size)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model_ss.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_ss.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
4622/4622 [==============================] - 1151s 248ms/step - loss: 0.6139 - accuracy: 0.7862 - val_loss: 0.5764 - val_accuracy: 0.7934 - lr: 0.0010
Epoch 2/10
4622/4622 [==============================] - 1198s 259ms/step - loss: 0.5524 - accuracy: 0.8037 - val_loss: 0.5602 - val_accuracy: 0.8006 - lr: 0.0010
Epoch 3/10
4622/4622 [==============================] - 1210s 262ms/step - loss: 0.5176 - accuracy: 0.8159 - val_loss: 0.5613 - val_accuracy: 0.8022 - lr: 0.0010
Epoch 4/10
4622/4622 [==============================] - 1282s 277ms/step - loss: 0.4850 - accuracy: 0.8280 - val_loss: 0.5707 - val_accuracy: 0.8028 - lr: 0.0010
Epoch 5/10
4622/4622 [==============================] - 1232s 267ms/step - loss: 0.4375 - accuracy: 0.8460 - val_loss: 0.5990 - val_accuracy: 0.8012 - lr: 5.0000e-04
Epoch 6/10
4622/4622 [==============================] - 1297s 281ms/step - loss: 0.4133 - accuracy: 0.8554 - val_loss: 0.6177 - val_accuracy: 0.7989 - lr: 5.0000e-04
Epoch 7/10
2311/2311

In [26]:
test_df['Review'] = test_df['Review'].fillna('')
X_test = test_df['Review'] 
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Predict ratings
predicted_labels = lstm_model_ss.predict(X_test_padded)
predicted_ratings = predicted_labels.argmax(axis=1)

# Decode numerical labels back to original categories
predicted_sentiments = label_encoder.inverse_transform(predicted_ratings)

# Combine predictions with IDs
predictions_df = pd.DataFrame({'id': test_df['id'], 'overall': predicted_sentiments})

# Save predictions
predictions_df.to_csv('lstm_ss.csv', index=False)

# Combine predictions with IDs
tm_df = pd.DataFrame({'id': test_df['id'], 'Review': test_df['Review'], 'overall': predicted_sentiments})

# Save predictions
tm_df.to_csv('lstm_ss_tm.csv', index=False)



3864/3864 [==============================] - 152s 39ms/step


In [ ]:
# # Define LSTM model with Word2Vec embeddings
# def LSTM_model2_with_Word2Vec(input_length, vocab_size, embedding_size, embedding_matrix):
#     model = Sequential()
#     model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
#     model.add(LSTM(200))
#     model.add(Dropout(0.35))
#     model.add(Dense(5, activation='softmax'))
#     model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

# # Create and compile the LSTM model with Word2Vec embeddings
# lstm_model2_with_word2vec = LSTM_model2_with_Word2Vec(max_len, vocab_size, embedding_size, embedding_matrix)

# # Train the model
# lstm_model2_with_word2vec.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# # Evaluate the model
# loss, accuracy = lstm_model2_with_word2vec.evaluate(X_test, y_test)
# print("Test Accuracy with Word2Vec embeddings:", accuracy)


Since the snowball stemmer ave us the highest accuracy with the LSTM and without word embeddings, we use the word2vec model along with LSTM only on the snowball stemmer preprocessed data

### LSTM + Word2Vec with Snowball Stemmer

In [29]:
from gensim.models import Word2Vec

# Split the data into features and target labels
X = train_df['ssprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Convert words to Word2Vec embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_size))
# Convert words to Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model with Word2Vec embeddings
def LSTM_model_with_Word2Vec_ss(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model_with_word2vec_ss = LSTM_model_with_Word2Vec_ss(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model_with_word2vec_ss.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_with_word2vec_ss.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


Epoch 1/5
4622/4622 [==============================] - 1438s 310ms/step - loss: 0.6153 - accuracy: 0.7870 - val_loss: 0.5743 - val_accuracy: 0.7962 - lr: 0.0010
Epoch 2/5
4622/4622 [==============================] - 1303s 282ms/step - loss: 0.5530 - accuracy: 0.8035 - val_loss: 0.5636 - val_accuracy: 0.8001 - lr: 0.0010
Epoch 3/5
4622/4622 [==============================] - 1272s 275ms/step - loss: 0.5190 - accuracy: 0.8153 - val_loss: 0.5636 - val_accuracy: 0.8031 - lr: 0.0010
Epoch 4/5
4622/4622 [==============================] - 1303s 282ms/step - loss: 0.4861 - accuracy: 0.8276 - val_loss: 0.5698 - val_accuracy: 0.8033 - lr: 0.0010
Epoch 5/5
2311/2311 [==============================] - 134s 58ms/step - loss: 0.5981 - accuracy: 0.8019
Test Accuracy with Word2Vec embeddings: 0.8019121885299683


In [30]:
test_df['Review'] = test_df['Review'].fillna('')
X_test = test_df['Review'] 
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Predict ratings
predicted_labels = lstm_model_with_word2vec_ss.predict(X_test_padded)
predicted_ratings = predicted_labels.argmax(axis=1)

# Decode numerical labels back to original categories
predicted_sentiments = label_encoder.inverse_transform(predicted_ratings)

# Combine predictions with IDs
predictions_df = pd.DataFrame({'id': test_df['id'], 'overall': predicted_sentiments})

# Save predictions
predictions_df.to_csv('lstm_w2v_ss.csv', index=False)

# Combine predictions with IDs
tm_df = pd.DataFrame({'id': test_df['id'], 'Review': test_df['Review'], 'overall': predicted_sentiments})

# Save predictions
tm_df.to_csv('lstm_w2v_ss_tm.csv', index=False)



3864/3864 [==============================] - 199s 51ms/step


In [ ]:
# def LSTM_model2_with_Word2Vec_ss(input_length, vocab_size, embedding_size, embedding_matrix):
#     model = Sequential()
#     model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
#     model.add(LSTM(200))
#     model.add(Dropout(0.35))
#     model.add(Dense(5, activation='softmax'))
#     model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

# # Create and compile the LSTM model with Word2Vec embeddings
# lstm_model2_with_word2vec_ss = LSTM_model2_with_Word2Vec_ss(max_len, vocab_size, embedding_size, embedding_matrix)

# # Train the model
# lstm_model2_with_word2vec_ss.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks_list)

# # Evaluate the model
# loss, accuracy = lstm_model2_with_word2vec_ss.evaluate(X_test, y_test)
# print("Test Accuracy with Word2Vec embeddings:", accuracy)

In [32]:
from gensim.models import Word2Vec

# Split the data into features and target labels
X = train_df['lemprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Convert words to Word2Vec embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_size))
# Convert words to Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model with Word2Vec embeddings
def LSTM_model_with_Word2Vec_lem(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model_with_word2vec_lem = LSTM_model_with_Word2Vec_lem(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model_with_word2vec_lem.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_with_word2vec_lem.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


Epoch 1/5
4622/4622 [==============================] - 1866s 403ms/step - loss: 0.6110 - accuracy: 0.7873 - val_loss: 0.5700 - val_accuracy: 0.7972 - lr: 0.0010
Epoch 2/5
4622/4622 [==============================] - 1850s 400ms/step - loss: 0.5424 - accuracy: 0.8070 - val_loss: 0.5606 - val_accuracy: 0.7997 - lr: 0.0010
Epoch 3/5
4622/4622 [==============================] - 1799s 389ms/step - loss: 0.5021 - accuracy: 0.8215 - val_loss: 0.5648 - val_accuracy: 0.8020 - lr: 0.0010
Epoch 4/5
4622/4622 [==============================] - 5980s 1s/step - loss: 0.4609 - accuracy: 0.8369 - val_loss: 0.5782 - val_accuracy: 0.8012 - lr: 0.0010
Epoch 5/5
2311/2311 [==============================] - 193s 84ms/step - loss: 0.6103 - accuracy: 0.8013
Test Accuracy with Word2Vec embeddings: 0.8013306856155396


In [33]:
test_df['Review'] = test_df['Review'].fillna('')
X_test = test_df['Review'] 
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len)

# Predict ratings
predicted_labels = lstm_model_with_word2vec_lem.predict(X_test_padded)
predicted_ratings = predicted_labels.argmax(axis=1)

# Decode numerical labels back to original categories
predicted_sentiments = label_encoder.inverse_transform(predicted_ratings)

# Combine predictions with IDs
predictions_df = pd.DataFrame({'id': test_df['id'], 'overall': predicted_sentiments})

# Save predictions
predictions_df.to_csv('lstm_w2v_lem.csv', index=False)

# Combine predictions with IDs
tm_df = pd.DataFrame({'id': test_df['id'], 'Review': test_df['Review'], 'overall': predicted_sentiments})

# Save predictions
tm_df.to_csv('lstm_w2v_lem_tm.csv', index=False)



3864/3864 [==============================] - 201s 52ms/step
